### 5.  Inferential statistics - select characteristics associated with infant survival.

In this section I will try to reduce the number of predictors to include in the training classifier.  to do this, I will compute the odds ratios and corresponding p-values for each of the binary categorical predictor variables in the data set.  The odds ratio approximates the probability of having a given factor (the predictor) for infants who lived to age 1 compared with those who died before that age.  Each of the predictors in this initial analysis indicates whether the newborn had any adverse health events at birth (e.g. seizure or injury) or congenital abnormalities.  This information is captured on birth certificates. Using these values, I will eliminate predictors that will not contribute much to classifying as they don't show much difference between infants who lived or did not live at age 1 year (indicated by the magnitude of the odds ratio) or where those odds ratios were not statistically significant at p=0.05.

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Read in a dataset with all linked infant deaths and a sample of 500 mis-matched birth-death records for preliminary analysis.
Read in the infant birth-death linked file for 2016-17.

In [2]:
eda5K = pd.read_csv(r'###\Py\Data\eda_df_5K.csv')
df1617_m = pd.read_csv(r'###\Py\Data\WAlinked1617_m.csv')

In [3]:
eda5K.shape, df1617_m.shape

((5631, 96), (631, 96))

Create a list of birth state file numbers (bsfn) for infants who died in 2016 or 2017.  These will be used to label the full birth data set for 2016-18 as living or dead at 1 year of age.

In [4]:
bsfnlist = df1617_m['bsfn'].tolist()

In [5]:
len(bsfnlist)

631

The next step is to create a column named 'alive' indicating the status of the infant at 1 year of age where 0 = dead and 1 = alive.

In [6]:
eda5K['alive']= np.where(eda5K['bsfn'].isin(bsfnlist), 0, 1)

In [7]:
eda5K.alive.value_counts(dropna=False)

1    5000
0     631
Name: alive, dtype: int64

#### Calculating odds ratios for different exposures and survival at 1 year

In [8]:
eda5K.dtypes

Match                    int64
Underlying COD Code     object
bMSB                    object
b_momrescity            object
b_momrescityfips       float64
b_momrescntyfips       float64
b_momrescountyl         object
b_momresstatefips       object
b_momreszip            float64
babnormcond             object
babnormcondunk          object
banencephaly            object
bbirplstatefips         object
bbirwtgr               float64
bcerttype               object
bchildab                object
bchildalive             object
bchildinjury            object
bchildtransfer          object
bcleftlip               object
bcleftpalate            object
bcongenanomaly          object
bcongenheart            object
bconghernia             object
bdob                    object
bdobd                  float64
bdobm                  float64
bdoby                  float64
bdownsyndrome           object
bfname                  object
bgastroschisis          object
bhypospadias            object
blimbdef

In [1]:
#eda5K.head()

###### Identifying factors that are related to infant survival at 1 year

Create a subset with only those variables that may be correlated with infant death.

In [10]:
df = eda5K.loc[:,['bMSB','babnormcond', 'banencephaly', 'bchildab', 'bchildinjury', 'bchildtransfer', 
                 'bcleftlip', 'bcleftpalate', 'bcongenanomaly', 'bcongenheart', 'bconghernia',
                'bdownsyndrome','bgastroschisis', 'bhypospadias', 'blimbdefect', 'bnicu','bomphalocele',
                'botherchrom', 'bseizure', 'bsurfactant', 'alive', 'Underlying COD Code']]


In [11]:
df.head()

,bMSB,babnormcond,banencephaly,bchildab,bchildinjury,bchildtransfer,bcleftlip,bcleftpalate,bcongenanomaly,bcongenheart,bconghernia,bdownsyndrome,bgastroschisis,bhypospadias,blimbdefect,bnicu,bomphalocele,botherchrom,bseizure,bsurfactant,alive,Underlying COD Code
0,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,V589
1,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,I634
2,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,C80
3,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,I251
4,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,W18


Identify and handle any null values.

In [12]:
df[df.isnull().any(axis=1)]

,bMSB,babnormcond,banencephaly,bchildab,bchildinjury,bchildtransfer,bcleftlip,bcleftpalate,bcongenanomaly,bcongenheart,bconghernia,bdownsyndrome,bgastroschisis,bhypospadias,blimbdefect,bnicu,bomphalocele,botherchrom,bseizure,bsurfactant,alive,Underlying COD Code
50,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,NaN
280,N,N,N,Y,N,N,N,N,Y,N,N,N,N,N,N,Y,N,N,N,N,0,NaN
341,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,NaN
539,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,NaN
876,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,NaN
1118,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,NaN
1122,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,NaN
1292,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,NaN
1419,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,Y,N,N,N,N,1,NaN
1450,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,NaN


In [31]:
df.isna().sum()

bMSB                    1
babnormcond             1
banencephaly            1
bchildab                1
bchildinjury            1
bchildtransfer          1
bcleftlip               1
bcleftpalate            1
bcongenanomaly          1
bcongenheart            1
bconghernia             1
bdownsyndrome           1
bgastroschisis          1
bhypospadias            1
blimbdefect             1
bnicu                   1
bomphalocele            1
botherchrom             1
bseizure                1
bsurfactant             1
alive                   0
Underlying COD Code    33
dtype: int64

For all categorical (Y, N) variables assign "N" when the value is missing.

In [13]:
catvars = ['bMSB','babnormcond','banencephaly','bchildab','bchildinjury','bchildtransfer',
           'bcleftlip','bcleftpalate','bcongenanomaly','bcongenheart','bconghernia',
           'bgastroschisis','bhypospadias','blimbdefect','bnicu','bomphalocele','botherchrom',
           'bseizure','bsurfactant','bdownsyndrome']

In [14]:
df[catvars]= df[catvars].fillna('N')
df.isna().sum()

bMSB                    0
babnormcond             0
banencephaly            0
bchildab                0
bchildinjury            0
bchildtransfer          0
bcleftlip               0
bcleftpalate            0
bcongenanomaly          0
bcongenheart            0
bconghernia             0
bdownsyndrome           0
bgastroschisis          0
bhypospadias            0
blimbdefect             0
bnicu                   0
bomphalocele            0
botherchrom             0
bseizure                0
bsurfactant             0
alive                   0
Underlying COD Code    33
dtype: int64

In [15]:
df = df.rename(columns={'Underlying COD Code': 'ucod'})

For underlying cause of death code ('ucod') fill NA values with code 'R95' which indicates an undefined condition. 

In [16]:
df['ucod'] = df['ucod'].fillna('R95')

df.isna().sum()

bMSB              0
babnormcond       0
banencephaly      0
bchildab          0
bchildinjury      0
bchildtransfer    0
bcleftlip         0
bcleftpalate      0
bcongenanomaly    0
bcongenheart      0
bconghernia       0
bdownsyndrome     0
bgastroschisis    0
bhypospadias      0
blimbdefect       0
bnicu             0
bomphalocele      0
botherchrom       0
bseizure          0
bsurfactant       0
alive             0
ucod              0
dtype: int64

Replace all unknown values ('U') with 'N' - there aren't many of these and it is likely that if any of these birth conditions were found they would be marked 'yes' on the birth record.

In [17]:
df = df.replace('U','N')
df.head()

,bMSB,babnormcond,banencephaly,bchildab,bchildinjury,bchildtransfer,bcleftlip,bcleftpalate,bcongenanomaly,bcongenheart,bconghernia,bdownsyndrome,bgastroschisis,bhypospadias,blimbdefect,bnicu,bomphalocele,botherchrom,bseizure,bsurfactant,alive,ucod
0,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,V589
1,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,I634
2,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,C80
3,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,I251
4,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,1,W18


In [37]:
df.shape

(5631, 22)

Convert all binary categorical values from 'Y', 'N' to 1, 0.

In [18]:
numvals = {'N': 0,
           'Y': 1}

In [19]:
df.replace(numvals, inplace=True)
df.head()

,bMSB,babnormcond,banencephaly,bchildab,bchildinjury,bchildtransfer,bcleftlip,bcleftpalate,bcongenanomaly,bcongenheart,bconghernia,bdownsyndrome,bgastroschisis,bhypospadias,blimbdefect,bnicu,bomphalocele,botherchrom,bseizure,bsurfactant,alive,ucod
0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,V589
1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,I634
2,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,C80
3,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,I251
4,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,W18
5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,C519
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,C259
7,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,I679
8,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,I251
9,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,F019


In this next step I fit a preliminary logistic regression model to see if I can eliminate some of the predictors from the final model I will use for classification.

In [74]:
logit = sm.Logit(df['alive'], df[catvars])

In [75]:
results=logit.fit(maxiter=200)


         Current function value: 0.282344
         Iterations: 200


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [76]:
print(results.summary())

                           Logit Regression Results                           
Dep. Variable:                  alive   No. Observations:                 5631
Model:                          Logit   Df Residuals:                     5611
Method:                           MLE   Df Model:                           19
Date:                Fri, 24 Jan 2020   Pseudo R-squ.:                  0.1951
Time:                        13:51:34   Log-Likelihood:                -1589.9
converged:                      False   LL-Null:                       -1975.3
Covariance Type:            nonrobust   LLR p-value:                3.236e-151
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
bMSB             -33.4097   1.18e+07  -2.83e-06      1.000   -2.32e+07    2.32e+07
babnormcond        0.8397      0.207      4.052      0.000       0.434       1.246
banencephaly     -29.3439   5.52e+05

In [79]:
print("ODDS RATIOS FOR PREDICTORS")
print(round(np.exp(results.params),2))

ODDS RATIOS FOR PREDICTORS
bMSB              0.00
babnormcond       2.32
banencephaly      0.00
bchildab          1.12
bchildinjury      0.83
bchildtransfer    0.33
bcleftlip         0.91
bcleftpalate      0.23
bcongenanomaly    6.23
bcongenheart      0.30
bconghernia       0.00
bgastroschisis    4.15
bhypospadias      1.08
blimbdefect       0.00
bnicu             0.50
bomphalocele      1.53
botherchrom       0.03
bseizure          0.13
bsurfactant       0.16
bdownsyndrome     0.33
dtype: float64


From the list of predictors in the logistic regression summary table above as well as the second table showing the odds ratios of each of the predictors I will include predictors that are statistically significantly correlated with the outcome (survival) at p=0.05 and that have larger odds ratios.  Based on these selection criteria I will include 'babnormcond' (indicating no abnormal condition of the newborn) and , 'bcongenanomaly' (indicating no congenital anomaly in the newborn) in the training model.